In [155]:
from tqdm import notebook
!pip install selenium
import pandas as pd    # csv
from selenium.webdriver.chrome.options import *
import time
import re
import base64    # encoder
# webdriver = 웹페이지를 제어하기 위한 모듈 >> 웹 그자체가 됨!
from selenium import webdriver as wb
# Keys = 컴퓨터의 키보드와 같은 역할 모듈, 컴퓨터가 쓸 키보드 
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup


chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

Defaulting to user installation because normal site-packages is not writeable


In [156]:
# 엑셀 파일 리뷰 부분 전처리하는 함수(.xlsx 제외하고 입력)
def do_preprocess(file_name):
    df = pd.read_excel(file_name + ".xlsx")
    not_using_text_list = ["한줄평", "병원리뷰", r"[0-9]."]
    
    for i in range(len(df)):
        temp = df.loc[i, '리뷰']
        temp = preprocess(temp)
        
        for not_using_text in not_using_text_list:
            temp = remove_text(temp, not_using_text)
            
        df.loc[i, '리뷰'] = temp
        
    df.to_excel(file_name + "전처리" + ".xlsx")

    
# 텍스트 전처리하는 함수
def preprocess(text):
    text= re.sub(r"[ㄱ-ㅎㅏ-ㅣ]+", ".", text)
    text = re.sub(r"[\`\~\@\#\$\%\^\&\*\(\)\-\_\=\+\<\>\;\:\'\"\[\]\{\}\\\/]", " ", text)
    text = re.sub(r"[^가-힣A-Za-z0-9\.\?\!]", " ", text)
    text = re.sub(r"\s*[\.]+", ".", text)
    text = re.sub(r"\s*[\?]+", "?", text)
    text = re.sub(r"\s*[\!]+", "!", text)
    text = re.sub(r"\s+", " ", text)
    text.strip()
    
    return text


# 불용어 제거하는 함수
def remove_text(original_text, not_using_text):
    result = re.sub(not_using_text, "", original_text)
    
    return result

In [166]:
def naver_place_crawl(driver, hospital):
    url = 'https://m.place.naver.com/place/list?level=top&entry=pll&x=null&y=null&query=' + hospital
    driver. get(url)
    time.sleep(1)
    
    hospitalUrl = driver.find_element(By.CLASS_NAME, '_3LMxZ')
    hospitalCode = hospitalUrl.get_attribute('href').split('/')[-1].split('?')[0]
    reviewUrl = 'https://pcmap.place.naver.com/hospital/' + hospitalCode + '/review/visitor'
    driver.get(reviewUrl)
    time.sleep(1)
    
    try:
        while True:
            moreButton = driver.find_element(By.CLASS_NAME, '_2kAri')
            moreButton.click()
            time.sleep(0.5)
    except:
        print('페이지 스크롤 끝')
        pass

    try:
        commentMoreBtn = driver.find_elements(By.CLASS_NAME, '_3_09q')
        for btn in commentMoreBtn:
            driver.execute_script('arguments[0].click();', btn)
#             btn.click()
    except:
        print('더보기 없음')
        pass
    
    
    html = driver.page_source
    soup = BeautifulSoup(html,'html.parser')

    result = []
    try:
        contents = soup.find_all('li', class_='_3l2Wz')

        for content in contents:
            try:
                date = content.find_all('span', class_='place_blind')[1].text
                if date == '최근 방문일':
                    date = content.find_all('span', class_='place_blind')[2].text
                elif date == '별점':
                    date = content.find_all('span', class_='place_blind')[2].text

                review = content.find('span', class_='WoYOw').text
                data = {
                    'review': review,
                    'date': date
                }
                result.append(data)
            except:
                pass
    except Exception as e:
        print(e)
        
    return result

In [170]:
result = naver_place_crawl(driver, '자연닮은부부한의원')

페이지 스크롤 끝


In [175]:
len('안녕하세요')

5

In [171]:
print(result)

[{'review': '직원분이랑 선생님 다 친절해요. 기계도 신식이였습니다. 통증이 있는 부위가 어디인지 정확히 알려주시고, 원인도 알려주셔서 좋았어요', 'date': '2022년 6월 29일 수요일'}, {'review': '좋아요', 'date': '2022년 6월 7일 화요일'}, {'review': '좋아요', 'date': '2022년 5월 28일 토요일'}, {'review': '5년 전 이 동네로 이사 온 이후부터 꾸준히 다니는 곳입니다. 매우 좋습니다.', 'date': '2022년 5월 19일 목요일'}, {'review': '굿', 'date': '2022년 5월 19일 목요일'}, {'review': '시설이 너무 깨끗하고 \n의사선생님께서 너무 친절하셔요 !!\n침도 안아프게 놓아주시고 너무 정성다해서 해주셔서\n최고인거 같아용 몸이 가볍네요 감사합니다~', 'date': '2022년 5월 12일 목요일'}, {'review': '친절해서 좋아요', 'date': '2022년 1월 25일 화요일'}, {'review': '좋아요', 'date': '2021년 11월 9일 화요일'}, {'review': '친절하고 좋아요', 'date': '2021년 11월 9일 화요일'}, {'review': '조앙ᆢㄷ', 'date': '2021년 10월 27일 수요일'}, {'review': 'ㅡ', 'date': '2021년 10월 23일 토요일'}, {'review': '치료는 나쁘지않은데 바빠서 필요한 치료는 다 안해주고 목, 어깨만 치료해준 느낌이에요 허리를 치료 못받고..ㅜ 추나도 예약해야 받을수 있다하고 암튼 그렇습니다', 'date': '2021년 7월 15일 목요일'}, {'review': '친절하세요', 'date': '2021년 6월 9일 수요일'}, {'review': '친절하시고 넘좋네요 ㅎㅎㅎ', 'date': '2021년 6월 1일 화요일'}, {'review': '자주 갑니다. 한의사 선생님 두 분 모두 친절하십니다. 침

In [173]:
# 현재 페이지 내 모든 병원 리스트에 저장하는 함수
def get_hospital_list(driver, hospital_kind):
    url = "https://m.place.naver.com/place/list?level=top&entry=pll&x=null&y=null&query=" + hospital_kind
    driver.get(url)
    do_scroll(driver)

    html = driver.page_source
    soup = BeautifulSoup(html,'html.parser')

    data = []

    elements  = driver.find_elements(By.CLASS_NAME, "place_bluelink")

    for element in elements:
        data.append(element.text)

    return data


# 스크롤 끝까지 내리는 함수
def do_scroll(driver):
    # Click on element inside the main content to switch focus
    driver.find_element(By.XPATH, '//div[@class="_2YHxL"]').click()

    for i in range(30):
        # Scroll page down
        driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.END)
        time.sleep(1)

In [151]:
result = []
try:
    contents = driver.find_elements(By.CLASS_NAME, '_3l2Wz')
    for content in contents:
        try:
            date = content.find_elements(By.CLASS_NAME, 'place_blind')[1].text
            if date == '최근 방문일':
                date = content.find_elements(By.CLASS_NAME, 'place_blind')[2].text
            elif date == '별점':
                date = content.find_elements(By.CLASS_NAME, 'place_blind')[2].text

            review = content.find_element(By.CLASS_NAME, 'WoYOw').text
            data = {
                'review': review,
                'date': date
            }
            result.append(data)
        except:
            pass
except Exception as e:
    print(e)


Message: no such element: Unable to locate element: {"method":"css selector","selector":".WoYOw"}
  (Session info: chrome=103.0.5060.114)
Stacktrace:
Backtrace:
	Ordinal0 [0x00426463+2188387]
	Ordinal0 [0x003BE461+1762401]
	Ordinal0 [0x002D3D78+802168]
	Ordinal0 [0x00301880+989312]
	Ordinal0 [0x00301B1B+989979]
	Ordinal0 [0x002F8071+950385]
	Ordinal0 [0x0031C824+1099812]
	Ordinal0 [0x002F7FF4+950260]
	Ordinal0 [0x0031C9F4+1100276]
	Ordinal0 [0x0032CC22+1166370]
	Ordinal0 [0x0031C5F6+1099254]
	Ordinal0 [0x002F6BE0+945120]
	Ordinal0 [0x002F7AD6+948950]
	GetHandleVerifier [0x006C71F2+2712546]
	GetHandleVerifier [0x006B886D+2652765]
	GetHandleVerifier [0x004B002A+520730]
	GetHandleVerifier [0x004AEE06+516086]
	Ordinal0 [0x003C468B+1787531]
	Ordinal0 [0x003C8E88+1805960]
	Ordinal0 [0x003C8F75+1806197]
	Ordinal0 [0x003D1DF1+1842673]
	BaseThreadInitThunk [0x777FFA29+25]
	RtlGetAppContainerNamedObjectPath [0x77A57A9E+286]
	RtlGetAppContainerNamedObjectPath [0x77A57A6E+238]



In [161]:
from bs4 import BeautifulSoup
import requests
import time

html = driver.page_source
soup = BeautifulSoup(html,'html.parser')


result = []
try:
    contents = soup.find_all('li', class_='_3l2Wz')
    
    for content in contents:
        try:
            date = content.find_all('span', class_='place_blind')[1].text
            if date == '최근 방문일':
                date = content.find_all('span', class_='place_blind')[2].text
            elif date == '별점':
                date = content.find_all('span', class_='place_blind')[2].text

            review = content.find('span', class_='WoYOw').text
            data = {
                'review': review,
                'date': date
            }
            result.append(data)
        except:
            pass
except Exception as e:
    print(e)

In [ ]:
boxes[2273].find('span', class_='WoYOw').text
boxes[0].find('div', class_='_29Yga').find_all('span', class_='utrsf')[1].find_all('span', class_='place_blind')[1].text

In [146]:
contents = driver.find_elements(By.CLASS_NAME, '_3l2Wz')

In [162]:
print(result)

[{'review': '2년에 한번씩 방문하는데 항상 친절하세요', 'date': '2022년 7월 18일 월요일'}, {'review': '친절해요', 'date': '2022년 7월 16일 토요일'}, {'review': '굿', 'date': '2022년 7월 16일 토요일'}, {'review': '굿', 'date': '2022년 7월 16일 토요일'}, {'review': '친구랑 동반내원하여 할인도 받고~~디자인도 잘나와서 기분도 좋구 ㅎㅎ선생님 친절하여 더더 좋아요^^', 'date': '2022년 7월 15일 금요일'}, {'review': '상담과 시술 모두 잘 해주셔서 만족합니다. 감사합니다.', 'date': '2022년 7월 14일 목요일'}, {'review': '1년 6개월마다 재시술 받으러 갑니다. 주변에서 눈썹 반영구 문신 한지도 모를 정도로 자연스럽게 잘 해줘서 너무 좋아요', 'date': '2022년 7월 13일 수요일'}, {'review': '친절하고 좋았는데요. 남자 눈썹이라 라인없이 자연스럽게 해달라고 했는데 그냥 여느 눈썹문신샵처럼 빼곡하게 원래 눈썹 모양까지 바뀌면서 됐어요...  홈페이지 처럼 자연스럽게 부분부분 채워넣는거라고 생각하고 가시는분들은 참고하세요..', 'date': '2022년 7월 12일 화요일'}, {'review': '친절하고 스타일에 맞게 잘 그려주셨어요', 'date': '2022년 7월 9일 토요일'}, {'review': '2년에 한번씩 방문해야될곳ㅋ', 'date': '2022년 7월 8일 금요일'}, {'review': '꼼꼼하게 잘 봐주셔서 만족해여~리터치때 조금만 손보면 되겠어용', 'date': '2022년 7월 7일 목요일'}, {'review': '굿', 'date': '2022년 7월 6일 수요일'}, {'review': '저랑 엄마랑 꾸준히 눈썹 문신하는 곳이에요ㅎㅎ', 'date': '2022년 7월 4일 월요일'}, {'review': '솔직히 비추 입니다 .. 아이

In [3]:
def naver_place_review(driver, hospital):
    try:
        # driver.get(url)
        # print('연결성공')
        result = []
        try:
            try:
                hospitalUrl = driver.find_elements(By.XPATH,
                                                   '//*[@id="loc-main-section-root"]/section/div/ul/li[contains(.,' + hospital + ')]')
            except:
                hospitalUrl = driver.find_elements(By.XPATH,
                                                   '//*[@id="loc-main-section-root"]/section/div/ul/li')
            if len(hospitalUrl) == 0:
                url = 'https://m.place.naver.com/place/list?level=top&entry=pll&x=null&y=null&query=' + hospital
                driver.get(url)
                # try:
                #     hospitalUrl = driver.find_element(By.XPATH,
                #                                       '//*[@id="_list_scroll_container"]/div/div/div[2]/ul/li/div[2]/a[1][contains(.,' + hospital + ')]')
                # except:
                #     hospitalUrl = driver.find_element(By.XPATH,
                #                                       '//*[@id="_list_scroll_container"]/div/div/div[2]/ul/li/div[2]/a[1]')

                hospitalUrl = driver.find_element(By.CLASS_NAME, '_3LMxZ')
                reviewDtail = hospitalUrl.get_attribute('href').split('/')[-1].split('?')[0]
                reviewUrl = 'https://pcmap.place.naver.com/hospital/' + reviewDtail + '/review/visitor'

                # script = f"window.open('{reviewUrl}');"
                # driver.execute_script(script)
                # driver.switch_to.window(driver.window_handles[-1])
                driver.get(reviewUrl)
                time.sleep(2)
                # time.sleep(3)
                moreButton = driver.find_elements(By.CSS_SELECTOR, 'div._2kAri')
                while len(moreButton) > 0:
                    moreButton[0].click()
                    time.sleep(1)
                    moreButton = driver.find_elements(By.CSS_SELECTOR, 'div._2kAri')
                try:
                    comentMoreBtn = driver.find_elements(By.CSS_SELECTOR, 'span._3_09q')
                    print(len(comentMoreBtn))
                    if len(comentMoreBtn) > 0:
                        for btn in comentMoreBtn:
                            driver.execute_script('arguments[0].click();', btn)
                            # btn.click()
                except:
                    print('없음')
                try:
                    # contents = driver.find_elements(By.CSS_SELECTOR, 'li._3l2Wz')
                    # for review in contents:
                    #     content = {
                    #         'review': review.find_element(By.CSS_SELECTOR, 'div.faZHB').text,
                    #         'date': review.find_element(By.XPATH, '//div[3]/span[1]/span[2]').text
                    #     }
                    #     result.append(content)
                    contents = driver.find_elements(By.CLASS_NAME, '_3l2Wz')
                    for review in contents:
                        try:
                            content = {
                                'review': review.find_element(By.CLASS_NAME, 'WoYOw').text,
                                # 'date': review.find_element(By.XPATH, '//div[3]/span[1]/span[2]').text
                                # 'date': review.find_element(By.CLASS_NAME, 'utrsf').find_elements(By.CLASS_NAME, 'place_blind')[1].text
                                'date': review.find_element(By.CLASS_NAME, '_29Yga').find_element(By.CLASS_NAME, 'utrsf').find_elements(By.CLASS_NAME, 'place_blind')[1].text
                            }
                            result.append(content)
                        except:
                            pass
                except:
                    print('없음')
                # print(result)
            else:
                for i in hospitalUrl:
                    reviewDtail = i.get_attribute('data-loc_plc-doc-id')
                    reviewUrl = 'https://pcmap.place.naver.com/hospital/' + reviewDtail + '/review/visitor'
                    script = f"window.open('{reviewUrl}');"
                    driver.execute_script(script)
                    driver.switch_to.window(driver.window_handles[1])
                    # time.sleep(3)
                    moreButton = driver.find_elements(By.CSS_SELECTOR, 'div._2kAri')
                    while len(moreButton) > 0:
                        moreButton[0].click()
                        time.sleep(1)
                        moreButton = driver.find_elements(By.CSS_SELECTOR, 'div._2kAri')
                    try:
                        comentMoreBtn = driver.find_elements(By.CSS_SELECTOR, 'span._3_09q')
                        print(len(comentMoreBtn))
                        if len(comentMoreBtn) > 0:
                            for btn in comentMoreBtn:
                                driver.execute_script('arguments[0].click();', btn)
                                # btn.click()
                    except:
                        print('없음')
                    try:
                        # contents = driver.find_elements(By.CSS_SELECTOR, 'li._3l2Wz')
                        # for review in contents:
                        #     content = {
                        #         'review': review.find_element(By.CSS_SELECTOR, 'div.faZHB').text,
                        #         'date': review.find_element(By.XPATH, '//div[3]/span[1]/span[2]').text
                        #     }
                        #     result.append(content)
                        contents = driver.find_elements(By.CLASS_NAME, '_3l2Wz')
                        for review in contents:
                            try:
                                content = {
                                    'review': review.find_element(By.CLASS_NAME, 'WoYOw').text,
                                    # 'date': review.find_element(By.CLASS_NAME, 'utrsf').find_elements(By.CLASS_NAME, 'place_blind')[1].text
                                    'date': review.find_element(By.CLASS_NAME, '_29Yga').find_element(By.CLASS_NAME, 'utrsf').find_elements(By.CLASS_NAME, 'place_blind')[1].text
                                }
                                result.append(content)
                            except:
                                pass
                    except:
                        print('없음')
                    print(result)
                    break
            return result
        except Exception as e:
            print('tistory Error:', e)
            result = 0
            return result
    except Exception as e:
        print('사이트 오류')
        result = 0
    return result

In [4]:
url = "https://www.naver.com/"

## 크롬 드라이버 실행
#  wb.Chrome() = 컴퓨터가 크롬창을 씀
driver = wb.Chrome()

# # 모두닥 페이지 이동
# driver.get(modoodoc_url)

In [5]:
result = naver_place_review(driver, '리앤채움의원')

206


In [20]:
result

[{'review': '친절해요', 'date': '2022년 7월 16일 토요일'},
 {'review': '굿', 'date': '2022년 7월 16일 토요일'},
 {'review': '굿', 'date': '2022년 7월 16일 토요일'},
 {'review': '친구랑 동반내원하여 할인도 받고~~디자인도 잘나와서 기분도 좋구 ㅎㅎ선생님 친절하여 더더 좋아요^^',
  'date': '2022년 7월 15일 금요일'},
 {'review': '상담과 시술 모두 잘 해주셔서 만족합니다. 감사합니다.', 'date': '2022년 7월 14일 목요일'},
 {'review': '1년 6개월마다 재시술 받으러 갑니다. 주변에서 눈썹 반영구 문신 한지도 모를 정도로 자연스럽게 잘 해줘서 너무 좋아요',
  'date': '2022년 7월 13일 수요일'},
 {'review': '친절하고 좋았는데요. 남자 눈썹이라 라인없이 자연스럽게 해달라고 했는데 그냥 여느 눈썹문신샵처럼 빼곡하게 원래 눈썹 모양까지 바뀌면서 됐어요... 홈페이지 처럼 자연스럽게 부분부분 채워넣는거라고 생각하고 가시는분들은 참고하세요..',
  'date': '2022년 7월 12일 화요일'},
 {'review': '친절하고 스타일에 맞게 잘 그려주셨어요', 'date': '2022년 7월 9일 토요일'},
 {'review': '2년에 한번씩 방문해야될곳ㅋ', 'date': '2022년 7월 8일 금요일'},
 {'review': '꼼꼼하게 잘 봐주셔서 만족해여~리터치때 조금만 손보면 되겠어용', 'date': '2022년 7월 7일 목요일'},
 {'review': '굿', 'date': '2022년 7월 6일 수요일'},
 {'review': '저랑 엄마랑 꾸준히 눈썹 문신하는 곳이에요ㅎㅎ', 'date': '2022년 7월 4일 월요일'},
 {'review': '솔직히 비추 입니다 .. 아이라인이 너무 가늘고 눈꼬리 약간 올려달랬는데 속눈썹 라인만 해주고 안해줬어요..\n간호사

In [21]:
len(result)

188

In [80]:
contents = driver.find_elements(By.CLASS_NAME, '_3l2Wz')

In [55]:
print(contents.text)

긍정의힘511
리뷰 151
팔로우
친절해요
최근 방문일
7.16.토
2022년 7월 16일 토요일
1번째 방문영수증


In [56]:
from bs4 import BeautifulSoup
import requests
import time

html = driver.page_source
soup = BeautifulSoup(html,'html.parser')

boxes = soup.find_all('li', class_='_3l2Wz')

In [64]:
len(boxes)

2274

In [66]:
boxes[2273].find('span', class_='WoYOw').text

'전화로 예약할시간 없어서 힘들었는데 네이버로 예약할수있어서 편하고 조아요~~눈썹도 만족합니다👍'

In [77]:
boxes[0].find('div', class_='_29Yga').find_all('span', class_='utrsf')[1].find_all('span', class_='place_blind')[1].text

IndexError: list index out of range

In [82]:
len(contents)

2274

In [81]:
print(contents[900].find_element(By.CLASS_NAME, 'faZHB').text)
# print(contents[200].find_element(By.CLASS_NAME, '_29Yga').find_elements(By.CLASS_NAME, 'place_blind')[1].text)

시술 하는분 진지하고 요구사항 적극적으로 잘 들어주셔서 감사했습니다


In [ ]:
result = []
for review in contents:
    try:
        content = {
            'review': review.find_element(By.CLASS_NAME, 'WoYOw').text,
            # 'date': review.find_element(By.CLASS_NAME, 'utrsf').find_elements(By.CLASS_NAME, 'place_blind')[1].text
            'date': review.find_element(By.CLASS_NAME, '_29Yga').find_elements(By.CLASS_NAME, 'place_blind')[1].text
        }
        result.append(content)
    except:
        pass

In [ ]:
result = []
for review in contents:
    try:
        content = {
            'review': review.find_element(By.CLASS_NAME, 'faZHB').text,
            # 'date': review.find_element(By.CLASS_NAME, 'utrsf').find_elements(By.CLASS_NAME, 'place_blind')[1].text
            'date': review.find_element(By.CLASS_NAME, '_29Yga').find_element(By.CLASS_NAME, 'utrsf').find_elements(By.CLASS_NAME, 'place_blind')[1].text
        }
    except Exception as e:
        print(e)
    
    result.append(content)

In [ ]:
len(result)

In [29]:
result

[['친절해요', '2022년 7월 16일 토요일'],
 ['굿', '2022년 7월 16일 토요일'],
 ['굿', '2022년 7월 16일 토요일'],
 ['친구랑 동반내원하여 할인도 받고~~디자인도 잘나와서 기분도 좋구 ㅎㅎ선생님 친절하여 더더 좋아요^^',
  '2022년 7월 15일 금요일'],
 ['상담과 시술 모두 잘 해주셔서 만족합니다. 감사합니다.', '2022년 7월 14일 목요일'],
 ['1년 6개월마다 재시술 받으러 갑니다. 주변에서 눈썹 반영구 문신 한지도 모를 정도로 자연스럽게 잘 해줘서 너무 좋아요',
  '2022년 7월 13일 수요일'],
 ['친절하고 좋았는데요. 남자 눈썹이라 라인없이 자연스럽게 해달라고 했는데 그냥 여느 눈썹문신샵처럼 빼곡하게 원래 눈썹 모양까지 바뀌면서 됐어요... 홈페이지 처럼 자연스럽게 부분부분 채워넣는거라고 생각하고 가시는분들은 참고하세요..',
  '2022년 7월 12일 화요일'],
 ['친절하고 스타일에 맞게 잘 그려주셨어요', '2022년 7월 9일 토요일'],
 ['2년에 한번씩 방문해야될곳ㅋ', '2022년 7월 8일 금요일'],
 ['꼼꼼하게 잘 봐주셔서 만족해여~리터치때 조금만 손보면 되겠어용', '2022년 7월 7일 목요일'],
 ['굿', '2022년 7월 6일 수요일'],
 ['저랑 엄마랑 꾸준히 눈썹 문신하는 곳이에요ㅎㅎ', '2022년 7월 4일 월요일'],
 ['솔직히 비추 입니다 .. 아이라인이 너무 가늘고 눈꼬리 약간 올려달랬는데 속눈썹 라인만 해주고 안해줬어요..\n간호사들 그닥 친절한것도 모르겠구요..\n발렛도 가능하다해서 갖고 갔는데 자리 없다고 실랑이하다가 구석에 셀프 주차하라더니3000원받더군요.. 시간없어서 돈 줬지만 주차비도 아니고 발렛비도 아니고… 이정도 가격이면 강남에 할곳 많아요~',
  '2022년 7월 4일 월요일'],
 ['1시간 반 정도 걸린다고 하셨는데 실제로 1시간 정도 걸린 것 같아요! 무난한 것 같습니다', '2022년 6월 30일 목요일